In [ ]:
import numpy as np
import pandas as pd
import plotly as plt
import plotly.express as px
import scipy
import math
from datetime import datetime

In [ ]:
df_matches = pd.read_csv("../data/cleaned/tennis_matches_cleaned.csv", parse_dates=['tourney_date'])
df_male_players = pd.read_csv("../data/cleaned/male_players_cleaned.csv")
df_female_players = pd.read_csv("../data/cleaned/female_players_cleaned.csv")

# Data integration

Create a dataframe with all the male matches and another one for the female matches

In [ ]:
df_male_players_tmp = pd.DataFrame()
df_male_players_tmp['player_name'] = df_male_players['name'] + ' ' + df_male_players['surname']
df_male_winners = df_male_players_tmp.join(df_matches.set_index('winner_name'), on='player_name', how='inner')
df_male_losers = df_male_players_tmp.join(df_matches.set_index('loser_name'), on='player_name', how='inner')
df_male_matches = pd.concat([df_male_winners, df_male_losers])

In [ ]:
df_male_matches

,player_name,Unnamed: 0,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,...,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,tourney_spectators,tourney_revenue,winner_name
643,Alexander Zverev,352,2019-580,Australian Open,Hard,128,G,2019-01-14,200,100644,...,13.0,6.0,12.0,4,6385.0,155,343.0,15583.0,2781000.18,NaN
643,Alexander Zverev,363,2019-580,Australian Open,Hard,128,G,2019-01-14,116,100644,...,13.0,2.0,8.0,4,6385.0,67,760.0,15583.0,2781000.18,NaN
643,Alexander Zverev,364,2019-580,Australian Open,Hard,128,G,2019-01-14,172,100644,...,26.0,12.0,17.0,4,6385.0,36,1120.0,15583.0,2781000.18,NaN
643,Alexander Zverev,1145,2019-M004,Acapulco,Hard,32,A,2019-02-25,298,100644,...,10.0,2.0,4.0,3,6475.0,64,797.0,3398.0,655207.08,NaN
643,Alexander Zverev,1149,2019-M004,Acapulco,Hard,32,A,2019-02-25,294,100644,...,10.0,0.0,2.0,3,6475.0,26,1443.0,3398.0,655207.08,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54476,Michele Vianello,94508,2021-9357,Forli CH,Clay,32,C,2021-06-14,243,104563,...,9.0,2.0,6.0,433,93.0,-1,NaN,3031.0,566031.89,Luca Vanni
54476,Michele Vianello,95338,2021-2120,Cordenons CH,Clay,32,C,2021-08-02,250,207608,...,10.0,2.0,7.0,291,195.0,-1,NaN,2876.0,543907.53,Timofey Skatov
54485,Ben Shelton,95003,2021-7316,Cary CH,Hard,32,C,2021-07-19,254,206499,...,9.0,3.0,5.0,413,100.0,1835,1.0,3074.0,583894.56,Aleksandar Kovacevic
54504,William Jansen,64393,2021-540,Wimbledon,Grass,128,G,2021-06-28,138,207760,...,13.0,8.0,16.0,454,87.0,-1,NaN,14566.0,5002793.97,Anton Matusevich


df_male_matches.describe()

In [ ]:
df_female_players_tmp = pd.DataFrame()
df_female_players_tmp['player_name'] = df_female_players['name'] + ' ' + df_female_players['surname']
df_female_winners = df_female_players_tmp.join(df_matches.set_index('winner_name'), on='player_name', how='inner')
df_female_losers = df_female_players_tmp.join(df_matches.set_index('loser_name'), on='player_name', how='inner')
df_female_matches = pd.concat([df_female_winners, df_female_losers])

In [ ]:
df_male_matches_reduced = df_male_losers[['player_name', 'winner_id', 'winner_ht', 'winner_age', 'tourney_date']]
df_male_matches_reduced[df_male_matches_reduced['player_name'] == 'Aaron Addison']

,player_name,winner_id,winner_ht,winner_age,tourney_date
31893,Aaron Addison,117357,-1,19,2016-10-24
31893,Aaron Addison,117360,-1,18,2016-02-01
31893,Aaron Addison,102864,193,-1,2016-01-11
31893,Aaron Addison,105429,-1,26,2016-10-31
31893,Aaron Addison,111806,-1,23,2020-01-13
31893,Aaron Addison,105349,-1,30,2019-10-21
31893,Aaron Addison,106172,-1,25,2019-01-07
31893,Aaron Addison,200614,-1,21,2019-10-28
31893,Aaron Addison,200640,-1,19,2018-10-22


- name
- hand
- sex
- nationality

- win percentage
- tot match
- number of won tournment

- year with more matches
- year with less matches
- year with more wins
- year with less wins

Reset the index of both the new data frames

In [ ]:
df_male_matches.reset_index(drop=True, inplace=True)
df_female_matches.reset_index(drop=True, inplace=True)

In [ ]:
df_player = pd.DataFrame()

list_of_male_names = df_male_matches['player_name'].unique()
print(f"Lunghezza lista: {len(list_of_male_names)}")
df_player['Name'] = list_of_male_names
df_player['Sex'] = 'M'
i = 0

for name in list_of_male_names:
    print(i)
    i+=1
    df_player_matches = df_matches[(df_matches['winner_name'] == name) | (df_matches['loser_name'] == name)]

    #player's hand
    df_h = [x for x in df_player_matches[(df_player_matches['winner_name'] == name) &  (df_player_matches['winner_hand'] != 'U')]['winner_hand']]
    if len(df_h) > 0:
        df_player.loc[df_player['Name'] == name, 'Hand'] = df_h[0]
    else:
        df_h = [x for x in df_player_matches[(df_player_matches['loser_name'] == name) &  (df_player_matches['loser_hand'] != 'U')]['loser_hand']]
        if len(df_h) > 0:
            df_player.loc[df_player['Name'] == name, 'Hand'] = df_h[0]
        else:
            df_player.loc[df_player['Name'] == name, 'Hand'] = 'U'

    #player's nationality
    df_h = [x for x in df_player_matches[(df_player_matches['winner_name'] == name) &  (df_player_matches['winner_ioc'] != None)]['winner_ioc']]
    if len(df_h) > 0:
        df_player.loc[df_player['Name'] == name, 'IOC'] = df_h[0]
    else:
        df_h = [x for x in df_player_matches[(df_player_matches['loser_name'] == name) &  (df_player_matches['loser_ioc'] != 'U')]['loser_ioc']]
        if len(df_h) > 0:
            df_player.loc[df_player['Name'] == name, 'IOC'] = df_h[0]
        else:
            df_player.loc[df_player['Name'] == name, 'IOC'] = 'Undefined'
    
    df_player.loc[df_player['Name'] == name, 'Won_Tournaments'] = df_player_matches[(df_player_matches['round'] == 'F') & (df_player_matches['winner_name'] == name)].shape[0]

    if df_player_matches.shape[0] != 0:
        df_player.loc[df_player['Name'] == name, 'Number_of_Matches'] = df_player_matches.shape[0]
        df_player.loc[df_player['Name'] == name, 'Win_Percentage'] = df_player_matches[df_player_matches['winner_name'] == name].shape[0]/df_player_matches.shape[0]
    
    match_per_year = df_male_matches[df_male_matches['player_name']==name]['tourney_date'].groupby(df_male_matches['tourney_date'].dt.year).count()
    if len(match_per_year) > 0:
        df_player.loc[df_player['Name'] == name, 'best_year_matches'] = match_per_year.idxmax()
        df_player.loc[df_player['Name'] == name, 'worst_year_matches'] = match_per_year.idxmin()

    wins_per_year = df_male_matches[df_male_matches['winner_name']==name]['tourney_date'].groupby(df_male_matches['tourney_date'].dt.year).count()
    if len(wins_per_year) > 0:
        df_player.loc[df_player['Name'] == name, 'best_year_wins'] = wins_per_year.idxmax()
        df_player.loc[df_player['Name'] == name, 'worst_year_wins'] = wins_per_year.idxmin()




NameError: name 'pd' is not defined

In [ ]:
df_player.to_csv("../data/integrations/matches_male_players.csv", index=False)

NameError: name 'df_player' is not defined

In [ ]:
df_player = pd.DataFrame()

list_of_female_names = df_female_matches['player_name'].unique()
print(list_of_female_names)
df_player['Name'] = list_of_female_names
df_player['Sex'] = 'F'
i = 0

for name in list_of_female_names:
    
    df_player_matches = df_matches[(df_matches['winner_name'] == name) | (df_matches['loser_name'] == name)]

    #player's hand
    df_h = [x for x in df_player_matches[(df_player_matches['winner_name'] == name) &  (df_player_matches['winner_hand'] != 'U')]['winner_hand']]
    if len(df_h) > 0:
        df_player.loc[df_player['Name'] == name, 'Hand'] = df_h[0]
    else:
        df_h = [x for x in df_player_matches[(df_player_matches['loser_name'] == name) &  (df_player_matches['loser_hand'] != 'U')]['loser_hand']]
        if len(df_h) > 0:
            df_player.loc[df_player['Name'] == name, 'Hand'] = df_h[0]
        else:
            df_player.loc[df_player['Name'] == name, 'Hand'] = 'U'

    #player's nationality
    df_h = [x for x in df_player_matches[(df_player_matches['winner_name'] == name) &  (df_player_matches['winner_ioc'] != None)]['winner_ioc']]
    if len(df_h) > 0:
        df_player.loc[df_player['Name'] == name, 'IOC'] = df_h[0]
    else:
        df_h = [x for x in df_player_matches[(df_player_matches['loser_name'] == name) &  (df_player_matches['loser_ioc'] != 'U')]['loser_ioc']]
        if len(df_h) > 0:
            df_player.loc[df_player['Name'] == name, 'IOC'] = df_h[0]
        else:
            df_player.loc[df_player['Name'] == name, 'IOC'] = 'Undefined'
    
    df_player.loc[df_player['Name'] == name, 'Won_Tournaments'] = df_player_matches[(df_player_matches['round'] == 'F') & (df_player_matches['winner_name'] == name)].shape[0]

    if df_player_matches.shape[0] != 0:
        df_player.loc[df_player['Name'] == name, 'Number_of_Matches'] = df_player_matches.shape[0]
        df_player.loc[df_player['Name'] == name, 'Win_Percentage'] = df_player_matches[df_player_matches['winner_name'] == name].shape[0]/df_player_matches.shape[0]

    match_per_year = df_female_matches[df_female_matches['player_name']==name]['tourney_date'].groupby(df_female_matches['tourney_date'].dt.year).count()
    if len(match_per_year) > 0:
        df_player.loc[df_player['Name'] == name, 'best_year_matches'] = match_per_year.idxmax()
        df_player.loc[df_player['Name'] == name, 'worst_year_matches'] = match_per_year.idxmin()

    wins_per_year = df_female_matches[df_female_matches['winner_name']==name]['tourney_date'].groupby(df_female_matches['tourney_date'].dt.year).count()
    if len(wins_per_year) > 0:
        df_player.loc[df_player['Name'] == name, 'best_year_wins'] = wins_per_year.idxmax()
        df_player.loc[df_player['Name'] == name, 'worst_year_wins'] = wins_per_year.idxmin()

['Mirjana Lucic' 'Maureen Drake' 'Serena Williams' ... 'Mariya Vyshkina'
 'Anna Sharandachenko' 'Mariya Sharandachenko']


In [ ]:
df_player.to_csv("../data/integrations/matches_female_players.csv", index=False)

## Start from here

In [ ]:
male_stats = pd.read_csv("../data/integrations/matches_male_players.csv")
female_stats = pd.read_csv("../data/integrations/matches_female_players.csv")

,Name,Sex,Hand,IOC,Won_Tournaments,Number_of_Matches,Win_Percentage,best_year_matches,worst_year_matches,best_year_wins,worst_year_wins
0,Alexander Zverev,M,R,GER,17.0,378.0,0.706349,2017.0,2020.0,2018.0,2020.0
1,Agustin Moreno,M,R,MEX,0.0,4.0,0.250000,2016.0,2016.0,2016.0,2016.0
2,Martin Damm,M,L,USA,0.0,7.0,0.285714,2021.0,2018.0,2021.0,2021.0
3,Gergely Kisgyorgy,M,R,HUN,0.0,2.0,0.500000,2016.0,2016.0,2016.0,2016.0
4,Frank Moser,M,R,GER,0.0,11.0,0.454545,2016.0,2017.0,2016.0,2016.0
...,...,...,...,...,...,...,...,...,...,...,...
3013,Lui Maxted,M,U,GBR,0.0,2.0,0.000000,2021.0,2021.0,NaN,NaN
3014,Lorenzo Angelini,M,U,ITA,0.0,1.0,0.000000,2021.0,2021.0,NaN,NaN
3015,Michele Vianello,M,R,ITA,0.0,2.0,0.000000,2021.0,2021.0,NaN,NaN
3016,William Jansen,M,L,GBR,0.0,1.0,0.000000,2021.0,2021.0,NaN,NaN


Idee:

- Dividere le statistiche dei player in base al tipo di surface dove giocano, come fatto nel sito https://www.atptour.com/en/players/novak-djokovic/d643/player-stats
- Fare un'analisi separata maschi/femmine e poi confrontarla con le performance che ci sono nei tornei misti (se ne troviamo)



### Dataframe Player_game_statistics

- % ace su tot punti al servizio
- % breakpoint salvati
- % first serve points won
- % vittorie per match lungo
- % sconfitta per match breve

In [ ]:
# ace = (#w_ace + #l_ace)/(#w_svpt + #l_svpt)
# bp = (#w_bpsaved + #l_bpsaved)/(#w_bpfaced + #w_bpfaced)
# first serve points won = (#w_1stWon + #l_1stWon)/(#w_1stln + #l_1stln:)
# average rank = (#winner_rank + #loser_rank)/(tot_match)

df_matches_cleaned = pd.concat([df_male_matches, df_female_matches])

#drop all the rows with at least one of the above attributes equal to null
player_game_statistics = df_matches_cleaned[(df_matches_cleaned['w_ace'].notna()) & (df_matches_cleaned['w_svpt'].notna()) & (df_matches_cleaned['w_bpSaved'].notna()) 
                                    & (df_matches_cleaned['w_bpFaced'].notna()) & (df_matches_cleaned['w_1stIn'].notna()) & (df_matches_cleaned['w_1stWon'].notna()) 
                                    & (df_matches_cleaned['l_ace'].notna()) & (df_matches_cleaned['l_svpt'].notna()) & (df_matches_cleaned['l_bpSaved'].notna()) 
                                    & (df_matches_cleaned['l_bpFaced'].notna()) & (df_matches_cleaned['l_1stIn'].notna()) & (df_matches_cleaned['l_1stWon'].notna())
                                    & (df_matches_cleaned['winner_rank'] > 0) & (df_matches_cleaned['loser_rank'] > 0)]

In [ ]:
player_match_statistics = pd.DataFrame()

player_names = list(set(list(player_game_statistics['winner_name'].unique()) + list(player_game_statistics['loser_name'].unique())))

player_match_statistics['Name'] = player_names

for player_name in player_names:
    df_player_winner = df_matches[df_matches['winner_name'] == player_name]
    df_player_loser = df_matches[df_matches['loser_name'] == player_name]
    
    #Ace
    ace_percent = (df_player_winner['w_ace'].sum() + df_player_winner['l_ace'].sum())/(df_player_winner['w_svpt'].sum() + df_player_winner['l_svpt'].sum())
    
    player_match_statistics.loc[player_match_statistics.Name == player_name, 'ace_percent'] = ace_percent

    #Break Point
    break_point = (df_player_winner['w_bpSaved'].sum() + df_player_winner['l_bpSaved'].sum())/(df_player_winner['w_bpFaced'].sum() + df_player_winner['l_bpFaced'].sum())
    
    player_match_statistics.loc[player_match_statistics.Name == player_name, 'break_point'] = break_point

    # First Serve Points Won
    fspw = (df_player_winner['w_1stWon'].sum() + df_player_winner['l_1stWon'].sum())/(df_player_winner['w_1stIn'].sum() + df_player_winner['l_1stIn'].sum())

    player_match_statistics.loc[player_match_statistics.Name == player_name, 'fspw'] = fspw
    
    # Average Rank
    average_rank = (df_player_winner['winner_rank'].sum() + df_player_winner['loser_rank'].sum())/(df_player_winner.shape[0] + df_player_loser.shape[0])

    player_match_statistics.loc[player_match_statistics.Name == player_name, 'average_rank'] = round(average_rank)

player_match_statistics


/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in double_scalars
  if sys.path[0] == '':
/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in double_scalars
/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in double_scalars
/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:27: RuntimeWarning: invalid value encountered in long_scalars


,Name,ace_percent,break_point,fspw,average_rank
0,NaN,NaN,NaN,NaN,NaN
1,Gonzalo Villanueva,0.034551,0.546371,0.645863,500.761905
2,Ben Patael,0.061041,0.581081,0.656706,466.525000
3,Filipe Brandao,0.032609,0.500000,0.519231,380.000000
4,Tim Puetz,0.091237,0.598425,0.714905,447.236842
...,...,...,...,...,...
3033,Jozef Kovalik,0.043836,0.590110,0.674781,215.178788
3034,Andres Molteni,0.034943,0.545205,0.670293,707.431034
3035,Kristian Lozan,0.065789,0.666667,0.670391,291.000000
3036,Thomas Statzberger,0.066339,0.481481,0.730290,355.888889


In [ ]:
player_match_statistics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3038 entries, 0 to 3037
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Name          3037 non-null   object 
 1   ace_percent   2246 non-null   float64
 2   break_point   2245 non-null   float64
 3   fspw          2246 non-null   float64
 4   average_rank  3037 non-null   float64
dtypes: float64(4), object(1)
memory usage: 118.8+ KB


In [ ]:
player_match_statistics = player_match_statistics[(player_match_statistics['ace_percent'].notna()) & (player_match_statistics['break_point'].notna()) 
                                                    & (player_match_statistics['fspw'].notna()) & (player_match_statistics['average_rank'].notna()) 
                                                    & (player_match_statistics['average_rank'] > 0)]
player_match_statistics

,Name,ace_percent,break_point,fspw,average_rank
1,Gonzalo Villanueva,0.034551,0.546371,0.645863,500.761905
2,Ben Patael,0.061041,0.581081,0.656706,466.525000
3,Filipe Brandao,0.032609,0.500000,0.519231,380.000000
4,Tim Puetz,0.091237,0.598425,0.714905,447.236842
6,Nicolas Kicker,0.040140,0.564485,0.659364,278.266667
...,...,...,...,...,...
3033,Jozef Kovalik,0.043836,0.590110,0.674781,215.178788
3034,Andres Molteni,0.034943,0.545205,0.670293,707.431034
3035,Kristian Lozan,0.065789,0.666667,0.670391,291.000000
3036,Thomas Statzberger,0.066339,0.481481,0.730290,355.888889


In [ ]:
player_match_statistics.to_csv('../data/integrations/player_match_statistics.csv')

### Dataframe Tourney_Statistics

In [ ]:

# tourney name (tourney names)
# spettatori medi (tourney spectators)
# rank medio giocatori (sum over all players in a tourney rank and divide )
# tourney_revenue (take cell with tourney revenue)
# tot_match

In [ ]:
#clean dataset of null values
df_matchey_tourney_stat_no_null = df_matches[(df_matches['tourney_name'].notna()) & (df_matches['tourney_spectators'].notna()) & (df_matches['tourney_id'].notna())]

In [ ]:
df_tourney_stats = df_matchey_tourney_stat_no_null[['tourney_name', 'tourney_id']].drop_duplicates().reset_index()

df_matches_group_by_tourney_mean = df_matchey_tourney_stat_no_null.groupby(['tourney_name', 'tourney_id']).mean().reset_index()
df_matches_group_by_tourney_count= df_matchey_tourney_stat_no_null.groupby(['tourney_name', 'tourney_id'], as_index=False).agg({'draw_size':'count'})

df_tourney_stats['avg_spect'] = df_matches_group_by_tourney_mean['tourney_spectators']

df_tourney_stats['avg_rank'] = (df_matches_group_by_tourney_mean['winner_rank'] + df_matches_group_by_tourney_mean['loser_rank'])/2

df_tourney_stats['avg_revenue'] =  df_matches_group_by_tourney_mean['tourney_revenue']

df_tourney_stats['tot_match'] = df_matches_group_by_tourney_count['draw_size']

df_tourney_stats

,index,tourney_name,tourney_id,avg_spect,avg_rank,avg_revenue,tot_match
0,0,Brisbane,2019-M020,2366.0,129.734694,449800.02,49
1,96,Doha,2019-0451,2773.0,180.406250,563339.81,48
2,139,Pune,2019-0891,447.0,104.125000,87559.10,16
3,178,Auckland,2019-0301,6175.0,173.620690,1576751.99,87
4,217,Sydney,2019-M001,2347.0,121.070652,446360.73,92
...,...,...,...,...,...,...,...
4878,185798,W25 Las Vegas NV,2020-W-ITF-USA-08A-2020,2366.0,117.794872,449800.02,39
4879,185853,W80 Macon GA,2020-W-ITF-USA-40A-2020,2773.0,134.230769,563339.81,39
4880,185908,W80 Tyler TX,2020-W-ITF-USA-42A-2020,3199.0,136.282051,763471.28,39
4881,185963,W100 Charleston SC,2020-W-ITF-USA-47A-2020,2462.0,93.576923,487848.44,39


In [ ]:
#drop rows with avg rank < 0
df_tourney_stats = df_tourney_stats[df_tourney_stats['avg_rank'] > 0]

In [ ]:
#box plot tot_match 
fig = px.box(df_tourney_stats, y="tot_match")
fig.show()

#drop all rows with tot_match > 76 and < 4
df_tourney_stats = df_tourney_stats[(df_tourney_stats['tot_match'] <= 76) & (df_tourney_stats['tot_match'] >= 4)]

df_tourney_stats

,index,tourney_name,tourney_id,avg_spect,avg_rank,avg_revenue,tot_match
0,0,Brisbane,2019-M020,2366.0,129.734694,449800.02,49
1,96,Doha,2019-0451,2773.0,180.406250,563339.81,48
2,139,Pune,2019-0891,447.0,104.125000,87559.10,16
9,898,Buenos Aires,2019-0506,3921.0,155.983051,560824.91,59
11,976,Rotterdam,2019-0407,3420.0,107.662791,660259.04,43
...,...,...,...,...,...,...,...
4878,185798,W25 Las Vegas NV,2020-W-ITF-USA-08A-2020,2366.0,117.794872,449800.02,39
4879,185853,W80 Macon GA,2020-W-ITF-USA-40A-2020,2773.0,134.230769,563339.81,39
4880,185908,W80 Tyler TX,2020-W-ITF-USA-42A-2020,3199.0,136.282051,763471.28,39
4881,185963,W100 Charleston SC,2020-W-ITF-USA-47A-2020,2462.0,93.576923,487848.44,39


In [ ]:
df_tourney_stats.to_csv('../data/integrations/tourney_statistics.csv')

### Dataframe TOP Player

In [ ]:
TOP = 50

In [ ]:
#Input:
# - df, the dataframe you want to work on
# - attr, the attribute you want to sum row by row
# - group_by, the attribute used to group the dataframe before summing (Here we sum rows with the same "group_by" value)
# - top_list, this other dataframe is left joined with df to restrict its result only to the top_list
# - top_list_attr, it's needed in the join function to know how's called the attribute

def top_players_feature(df, attr, group_by, top_list, top_list_attr):
    #count 
    tmp = pd.DataFrame(df[df[attr]>=0].groupby(group_by, as_index=False)[attr].sum())
    #intersection with top_list
    tmp = top_list.join(tmp.set_index(group_by), on=top_list_attr, how='left')
    
    return tmp

#Function to generate the sum of a column, with a <, grouped by the attribute given in input

def sum_by_surface(df, attr, group_by, top_list, top_list_attr):
    #count 
    tmp = pd.DataFrame(df[df[attr]>=0].groupby([group_by, 'surface'], as_index=False)[attr].sum())
    #intersection with top_list
    tmp = top_list.join(tmp.set_index(group_by), on=top_list_attr, how='left')
    
    return [tmp[tmp['surface'] == 'Clay'], tmp[tmp['surface'] == 'Grass'], tmp[tmp['surface'] == 'Hard'], tmp[tmp['surface'] == 'Carpet']]

In [ ]:
# take the top players according to number of won tournament
df_top_male_players = pd.DataFrame(male_stats.sort_values(by='Won_Tournaments', ascending=False).head(TOP)['Name'])

#iterating for every player
#for name,index in df_top_male_players.iterrows():
#df_matches.groupby('winner_name').count()['w_ace'] 

df_top_male_player_matches_win = df_matches.join(df_top_male_players.set_index('Name'), on='winner_name', how='inner') #Ci sono partite in cui i winner sono loser e non vengono considerate
df_top_male_player_matches_los = df_matches.join(df_top_male_players.set_index('Name'), on='loser_name', how='inner') #Ci sono partite in cui i winner sono loser e non vengono considerate
df_top_male_player_matches  = pd.concat([df_top_male_player_matches_win, df_top_male_player_matches_los])
#df_top_male_players['w_ace'] = sum_by_condition(df_top_male_player_matches, 'w_ace', 'winner_name', df_top_male_players, 'Name')['w_ace']
# Ciclo for su lista di attributi che chiama sum_by_condition

attrs = ['w_ace', 'w_df', 'w_svpt', 'w_1stIn', 'w_1stWon', 'w_2ndWon', 'w_SvGms', 'w_bpSaved', 'w_bpFaced']

for attr in attrs:
    #df_top_male_players[attr] = top_players_feature(df_top_male_player_matches, attr, 'winner_name', df_top_male_players, 'Name')[attr]
    result = sum_by_surface(df_top_male_player_matches, attr, 'winner_name', df_top_male_players, 'Name')
    df_top_male_players[attr + '_clay'] = result[0][attr]
    df_top_male_players[attr + '_clay'].fillna(0, inplace=True)
    df_top_male_players[attr + '_grass'] = result[1][attr]
    df_top_male_players[attr + '_grass'].fillna(0, inplace=True)
    df_top_male_players[attr + '_hard'] = result[2][attr]
    df_top_male_players[attr + '_hard'].fillna(0, inplace=True)
    df_top_male_players[attr + '_carpet'] = result[2][attr]
    df_top_male_players[attr + '_carpet'].fillna(0, inplace=True)

df_top_male_players

,Name,w_ace_clay,w_ace_grass,w_ace_hard,w_ace_carpet,w_df_clay,w_df_grass,w_df_hard,w_df_carpet,w_svpt_clay,...,w_SvGms_hard,w_SvGms_carpet,w_bpSaved_clay,w_bpSaved_grass,w_bpSaved_hard,w_bpSaved_carpet,w_bpFaced_clay,w_bpFaced_grass,w_bpFaced_hard,w_bpFaced_carpet
192,Novak Djokovic,312.0,107.0,460.0,460.0,244.0,147.0,569.0,569.0,9237.0,...,2583.0,2583.0,445.0,152.0,588.0,588.0,654.0,208.0,859.0,859.0
154,Rafael Nadal,330.0,35.0,451.0,451.0,269.0,32.0,336.0,336.0,10917.0,...,2139.0,2139.0,537.0,47.0,450.0,450.0,760.0,64.0,647.0,647.0
0,Alexander Zverev,245.0,11.0,276.0,276.0,389.0,88.0,781.0,781.0,8221.0,...,2535.0,2535.0,310.0,75.0,642.0,642.0,508.0,102.0,915.0,915.0
35,Roger Federer,52.0,107.0,280.0,280.0,27.0,64.0,278.0,278.0,1563.0,...,1943.0,1943.0,57.0,150.0,375.0,375.0,81.0,196.0,529.0,529.0
766,Daniil Medvedev,89.0,44.0,416.0,416.0,100.0,98.0,790.0,790.0,2496.0,...,2947.0,2947.0,118.0,84.0,710.0,710.0,180.0,123.0,1030.0,1030.0
679,Dominic Thiem,471.0,15.0,293.0,293.0,412.0,63.0,433.0,433.0,12133.0,...,2241.0,2241.0,517.0,47.0,542.0,542.0,765.0,60.0,805.0,805.0
188,Andy Murray,72.0,40.0,249.0,249.0,84.0,38.0,318.0,318.0,2609.0,...,1394.0,1394.0,126.0,88.0,343.0,343.0,181.0,128.0,522.0,522.0
1051,Andrey Rublev,136.0,34.0,389.0,389.0,135.0,32.0,560.0,560.0,5121.0,...,2411.0,2411.0,261.0,42.0,629.0,629.0,405.0,68.0,901.0,901.0
1366,Ugo Humbert,25.0,10.0,201.0,201.0,24.0,75.0,310.0,310.0,712.0,...,1340.0,1340.0,25.0,88.0,362.0,362.0,48.0,118.0,480.0,480.0
546,James Duckworth,20.0,19.0,138.0,138.0,27.0,26.0,250.0,250.0,713.0,...,1330.0,1330.0,49.0,63.0,314.0,314.0,62.0,76.0,425.0,425.0


In [ ]:
df_top_male_players.to_csv("../data/integrations/male_winners_stats_top.csv", index=False)

In [ ]:
male_stats

,Name,Sex,Hand,IOC,Won_Tournaments,Number_of_Matches,Win_Percentage,best_year_matches,worst_year_matches,best_year_wins,worst_year_wins
0,Alexander Zverev,M,R,GER,17.0,378.0,0.706349,2017.0,2020.0,2018.0,2020.0
1,Agustin Moreno,M,R,MEX,0.0,4.0,0.250000,2016.0,2016.0,2016.0,2016.0
2,Martin Damm,M,L,USA,0.0,7.0,0.285714,2021.0,2018.0,2021.0,2021.0
3,Gergely Kisgyorgy,M,R,HUN,0.0,2.0,0.500000,2016.0,2016.0,2016.0,2016.0
4,Frank Moser,M,R,GER,0.0,11.0,0.454545,2016.0,2017.0,2016.0,2016.0
...,...,...,...,...,...,...,...,...,...,...,...
3013,Lui Maxted,M,U,GBR,0.0,2.0,0.000000,2021.0,2021.0,NaN,NaN
3014,Lorenzo Angelini,M,U,ITA,0.0,1.0,0.000000,2021.0,2021.0,NaN,NaN
3015,Michele Vianello,M,R,ITA,0.0,2.0,0.000000,2021.0,2021.0,NaN,NaN
3016,William Jansen,M,L,GBR,0.0,1.0,0.000000,2021.0,2021.0,NaN,NaN


In [ ]:
# DO the same for womens
# take the top players according to number of won tournament
df_top_female_players = pd.DataFrame(female_stats.sort_values(by='Won_Tournaments', ascending=False).head(TOP)['Name'])

#iterating for every player
#for name,index in df_top_female_players.iterrows():
#df_matches.groupby('winner_name').count()['w_ace'] 

df_top_female_player_matches_win = df_matches.join(df_top_female_players.set_index('Name'), on='winner_name', how='inner') #Ci sono partite in cui i winner sono loser e non vengono considerate
df_top_female_player_matches_los = df_matches.join(df_top_female_players.set_index('Name'), on='loser_name', how='inner') #Ci sono partite in cui i winner sono loser e non vengono considerate
df_top_female_player_matches  = pd.concat([df_top_female_player_matches_win, df_top_female_player_matches_los])
#df_top_female_players['w_ace'] = sum_by_condition(df_top_female_player_matches, 'w_ace', 'winner_name', df_top_female_players, 'Name')['w_ace']
# Ciclo for su lista di attributi che chiama sum_by_condition

attrs = ['w_ace', 'w_df', 'w_svpt', 'w_1stIn', 'w_1stWon', 'w_2ndWon', 'w_SvGms', 'w_bpSaved', 'w_bpFaced']

#sum_by_condition(df_top_female_player_matches, 'w_svpt', 'winner_name', df_top_female_players, 'Name')

#sum_by_surface(df_top_female_player_matches, 'w_svpt', 'winner_name', df_top_female_players, 'Name', 'Clay')

for attr in attrs:
    #df_top_female_players[attr] = top_players_feature(df_top_female_player_matches, attr, 'winner_name', df_top_female_players, 'Name')[attr]
    result = sum_by_surface(df_top_female_player_matches, attr, 'winner_name', df_top_female_players, 'Name')
    df_top_female_players[attr + '_clay'] = result[0][attr]
    df_top_female_players[attr + '_clay'].fillna(0, inplace=True)
    df_top_female_players[attr + '_grass'] = result[1][attr]
    df_top_female_players[attr + '_grass'].fillna(0, inplace=True)
    df_top_female_players[attr + '_hard'] = result[2][attr]
    df_top_female_players[attr + '_hard'].fillna(0, inplace=True)
    df_top_female_players[attr + '_carpet'] = result[2][attr]
    df_top_female_players[attr + '_carpet'].fillna(0, inplace=True)

df_top_female_players

,Name,w_ace_clay,w_ace_grass,w_ace_hard,w_ace_carpet,w_df_clay,w_df_grass,w_df_hard,w_df_carpet,w_svpt_clay,...,w_SvGms_hard,w_SvGms_carpet,w_bpSaved_clay,w_bpSaved_grass,w_bpSaved_hard,w_bpSaved_carpet,w_bpFaced_clay,w_bpFaced_grass,w_bpFaced_hard,w_bpFaced_carpet
722,Fernanda Brito,2.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,166.0,...,0.0,0.0,5.0,0.0,0.0,0.0,13.0,0.0,0.0,0.0
118,Arantxa Rus,57.0,23.0,90.0,90.0,98.0,19.0,114.0,114.0,2254.0,...,408.0,408.0,128.0,25.0,166.0,166.0,216.0,35.0,261.0,261.0
434,Nuria Parrizas Diaz,2.0,6.0,1.0,1.0,23.0,8.0,3.0,3.0,1135.0,...,21.0,21.0,67.0,3.0,5.0,5.0,115.0,6.0,12.0,12.0
1494,Tamara Zidansek,69.0,1.0,43.0,43.0,152.0,6.0,69.0,69.0,4226.0,...,306.0,306.0,284.0,10.0,142.0,142.0,481.0,27.0,228.0,228.0
1578,Ylona Georgiana Ghioroaie,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
248,Ashleigh Barty,130.0,71.0,341.0,341.0,136.0,117.0,464.0,464.0,3280.0,...,1680.0,1680.0,163.0,121.0,555.0,555.0,244.0,175.0,859.0,859.0
272,Elina Svitolina,101.0,41.0,506.0,506.0,134.0,44.0,467.0,467.0,3770.0,...,1990.0,1990.0,227.0,64.0,701.0,701.0,375.0,99.0,1163.0,1163.0
1523,Aryna Sabalenka,79.0,48.0,371.0,371.0,143.0,91.0,667.0,667.0,2465.0,...,1598.0,1598.0,159.0,110.0,572.0,572.0,234.0,166.0,877.0,877.0
176,Karolina Pliskova,102.0,61.0,356.0,356.0,187.0,109.0,541.0,541.0,3804.0,...,1822.0,1822.0,191.0,113.0,651.0,651.0,299.0,163.0,972.0,972.0
142,Simona Halep,145.0,46.0,250.0,250.0,208.0,53.0,306.0,306.0,5456.0,...,1392.0,1392.0,303.0,89.0,446.0,446.0,514.0,143.0,770.0,770.0


In [ ]:
df_top_female_players.to_csv("../data/integrations/female_winners_stats_top.csv", index=False)

### TODO: for losers?

### New Features analysis